In [ ]:
import os
import sys
import random
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import Model, load_model
from keras.layers import Input, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
im_width = 512
im_height = 512
im_chan = 1

path_train = '/kaggle/input/small-dataset/Train/'
path_test = '/kaggle/input/small-dataset/Test/'

train_ids = next(os.walk(path_train+"images"))[2]
test_ids = next(os.walk(path_test+"images"))[2]

In [ ]:
X_train = np.zeros((len(train_ids), im_height, im_width, im_chan), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), im_height, im_width, 1), dtype=bool)

print('Getting and resizing train images and masks ... ')

sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    path = path_train
    img = load_img(path + '/images/' + id_)
    
    x = img_to_array(img)[:,:,1]
    x = np.expand_dims(x, axis=-1)
    X_train[n] = x
    
    
    mask = img_to_array(load_img(path + '/masks/' + id_))[:,:,1]
    mask = np.expand_dims(mask, axis=-1)
    Y_train[n] = mask
    

print('Done!')

In [ ]:
ix = random.randint(0, len(train_ids))
plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.imshow(np.dstack((X_train[ix],X_train[ix],X_train[ix])))
plt.title("Image")

plt.subplot(1, 3, 2)
tmp = np.squeeze(Y_train[ix]).astype(np.float32)
plt.imshow(np.dstack((tmp,tmp,tmp)))
plt.title("Ground truth")

plt.show()

In [ ]:
from tensorflow.keras import metrics
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable()
class MeanIoUMetric(metrics.Metric):
    
    def __init__(self, num_classes, name='mean_iou', **kwargs):
        super(MeanIoUMetric, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes  
        self.iou_metric = metrics.MeanIoU(num_classes=num_classes) 

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_ = tf.cast(y_pred > 0.5, tf.int32) 
        self.iou_metric.update_state(y_true, y_pred_)

    def result(self):
        return self.iou_metric.result()

    def reset_state(self):
        self.iou_metric.reset_state()
        
    @classmethod
    def from_config(cls, config):
        return cls(**config) 

    def get_config(self):
        config = super().get_config()
        config.update({"num_classes": self.num_classes}) 
        return config
    
    
mean_iou_metric = MeanIoUMetric(num_classes=2)

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, BatchNormalization

def conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu", kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu", kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    return x

def up_conv(x, filters, kernel_size=(2, 2), strides=(2, 2), padding="same"):
    x = Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    return x

def unet_plus_plus(input_size=(512, 512, 1), num_classes=1, deep_supervision=False):
    inputs = Input(input_size)
    s = Lambda(lambda x: x / 255)(inputs)  

    x0_0 = conv_block(s, 8) 
    x1_0 = conv_block(MaxPooling2D((2, 2))(x0_0), 16)
    x0_1 = conv_block(concatenate([x0_0, up_conv(x1_0, 8)], axis=3), 8)

    x2_0 = conv_block(MaxPooling2D((2, 2))(x1_0), 32)
    x1_1 = conv_block(concatenate([x1_0, up_conv(x2_0, 16)], axis=3), 16)
    x0_2 = conv_block(concatenate([x0_0, x0_1, up_conv(x1_1, 8)], axis=3), 8)

    x3_0 = conv_block(MaxPooling2D((2, 2))(x2_0), 64)
    x2_1 = conv_block(concatenate([x2_0, up_conv(x3_0, 32)], axis=3), 32)
    x1_2 = conv_block(concatenate([x1_0, x1_1, up_conv(x2_1, 16)], axis=3), 16)
    x0_3 = conv_block(concatenate([x0_0, x0_1, x0_2, up_conv(x1_2, 8)], axis=3), 8)
  
    x4_0 = conv_block(MaxPooling2D((2, 2))(x3_0), 128)
    x3_1 = conv_block(concatenate([x3_0, up_conv(x4_0, 64)], axis=3), 64)
    x2_2 = conv_block(concatenate([x2_0, x2_1, up_conv(x3_1, 32)], axis=3), 32)
    x1_3 = conv_block(concatenate([x1_0, x1_1, x1_2, up_conv(x2_2, 16)], axis=3), 16)
    x0_4 = conv_block(concatenate([x0_0, x0_1, x0_2, x0_3, up_conv(x1_3, 8)], axis=3), 8)

    if deep_supervision:
        outputs = [Conv2D(num_classes, (1, 1), activation="sigmoid", name=f"output_{i}")(x) for i, x in enumerate([x0_1, x0_2, x0_3, x0_4])]
        model = Model(inputs=[inputs], outputs=outputs)
        model.summary()
    else:
        outputs = Conv2D(num_classes, (1, 1), activation="sigmoid", name="output")(x0_4)
        model = Model(inputs=[inputs], outputs=[outputs])
        model.summary()
        
    return model
model = unet_plus_plus()
model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=[ mean_iou_metric])

In [ ]:
earlystopper = EarlyStopping(patience=10, verbose=2)
checkpointer = ModelCheckpoint('UNET++.keras', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.2, batch_size=16, epochs=150, 
                    callbacks=[earlystopper, checkpointer])

In [ ]:
import matplotlib.pyplot as plt


epochs = range(1, len(results.history['loss']) + 1)
loss = results.history['loss']
iou = results.history['mean_iou']
val_iou = results.history['val_mean_iou']


loss_Training_color = 'red' 
Mean_IoU_color = 'orange'
Validation_Mean_IoU_color = 'blue'  


fig, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()


ax1.plot(epochs, loss, color=loss_Training_color, linewidth=2, label='Training Loss')
ax2.plot(epochs, iou, color=Mean_IoU_color, linewidth=2, label='Training Mean IoU')
ax2.plot(epochs, val_iou, color=Validation_Mean_IoU_color, linestyle='--', linewidth=2, label='Validation Mean IoU')


ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12, color=loss_color)
ax2.set_ylabel('Mean IoU', fontsize=12, color=iou_color)

ax1.tick_params(axis='y', labelcolor=loss_color)
ax2.tick_params(axis='y', labelcolor=iou_color)

ax1.tick_params(axis='both', labelsize=10)
ax2.tick_params(axis='y', labelsize=10)
ax1.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)


ax1.set_ylim(0, 1)  
ax2.set_ylim(0, 1) 


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left', fontsize=9, frameon=False)


plt.tight_layout()


plt.savefig('UNET++.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Get and resize test images and masks
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan), dtype=np.uint8)
Y_test = np.zeros((len(test_ids), im_height, im_width, 1), dtype=bool) 



print('Getting and resizing test images and masks ... ')
sys.stdout.flush()

for n, id_ in tqdm_notebook(enumerate(test_ids), total=len(test_ids)):
    path = path_test
    img = load_img(path + '/images/' + id_)
    x = img_to_array(img)[:,:,1]
    x = np.expand_dims(x, axis=-1)
    X_test[n] = x

  
    mask = img_to_array(load_img(path + '/masks/' + id_))[:,:,1]
    mask = np.expand_dims(mask, axis=-1)
    Y_test[n] = mask

print('Done!')

In [ ]:
model = tf.keras.models.load_model(
    '/kaggle/working/UNET++.keras',
    custom_objects={'MeanIoUMetric': MeanIoUMetric} ,safe_mode=False
)


preds_test = model.predict(X_test, verbose=1)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
#Perform a sanity check on some random Test Samples

import matplotlib.pyplot as plt
import numpy as np
import random

 
plt.style.use('seaborn-whitegrid') 

fig, axes = plt.subplots(3, 3, figsize=(15, 12))  
fig.subplots_adjust(hspace=0.3, wspace=-0.5)

for i in range(3):
    if i ==0 :
        axes[i, 0].set_title("Image", fontsize=16) 
        axes[i, 1].set_title("Ground Truth", fontsize=16)
        axes[i, 2].set_title("Predicted", fontsize=16)
        
        
    ix = random.randint(0, len(preds_test_t) - 1) 

    # Image
    axes[i, 0].imshow(np.dstack((X_test[ix], X_test[ix], X_test[ix])))
    
    axes[i, 0].axis('off') 

    
    # Ground Truth
    im_gt = axes[i, 1].imshow(Y_test[ix], cmap='magma')
    
    axes[i, 1].axis('off')
    

    
    # Prediction
    im_pred = axes[i, 2].imshow(preds_test_t[ix], cmap='magma')
    
    axes[i, 2].axis('off')
    

plt.savefig("prediction_visualization.png", dpi=600) 
plt.show()